In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


caffe_root = '~/caffe/' 
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


DEMO_DIR = './DemoDir'

categories = [ 'Angry' , 'Disgust' , 'Fear' , 'Happy'  , 'Neutral' ,  'Sad' , 'Surprise']

/home/priyanka/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def showimage(im):
    if im.ndim == 3:
        im = im[:, :, ::-1]
    plt.set_cmap('jet')
    plt.imshow(im,vmin=0, vmax=0.2)
    

def vis_square(data, padsize=1, padval=0):
    data -= data.min()
    data /= data.max()
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))
    
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    
    showimage(data)

In [6]:
cur_net_dir = 'VGG_S_rgb'

mean_filename=os.path.join(DEMO_DIR,cur_net_dir,'mean.binaryproto')
proto_data = open(mean_filename, "rb").read()
a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data)
mean  = caffe.io.blobproto_to_array(a)[0]

net_pretrained = os.path.join(DEMO_DIR,cur_net_dir,'EmotiW_VGG_S.caffemodel')
net_model_file = os.path.join(DEMO_DIR,cur_net_dir,'deploy.prototxt')
VGG_S_Net = caffe.Classifier(net_model_file, net_pretrained,
                       mean=mean,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

input_image = caffe.io.load_image(os.path.join(DEMO_DIR,cur_net_dir,'cohn-kanade-images/S014/004/S014_004_00000019.png'))
prediction = VGG_S_Net.predict([input_image],oversample=False)
print 'predicted category is {0}'.format(categories[prediction.argmax()])

predicted category is Disgust


In [57]:
import os
from os import listdir
from cd import cd

def get_all_folders(folder_path):
    with cd(folder_path):
        all_folders = os.listdir('.')
        directory_names = []
        for directory in all_folders:
            if directory[0] == '.':
                continue
            else:
                directory_names.append(directory)
        return directory_names
    
labels = list()
images = list()
valid_images = []

def grab_data(Folder):
    counter = 0
    
    ck_path = os.getcwd() + '/DemoDir/VGG_S_rgb/' + Folder + '/'
    s_folders = get_all_folders(ck_path)
    for folder in s_folders:
        subdir_path = ck_path + folder + '/'
        zero_folders = get_all_folders(subdir_path)
        for zero in zero_folders:
            img_level_path = subdir_path + zero + '/'
            imgs = get_all_folders(img_level_path)
            if Folder == 'Emotion': #LABELS
                if len(imgs) > 0: #has an emotion label
                    last_img_path = img_level_path + imgs[-1]
                    f = open(last_img_path, 'r')
                    emotion = f.read()
                    labels.append(emotion[3]) #emotion[3] is the number denoting emotion
                else: #has no emotion label
                    labels.append(-1)
            else: #FEATURES
                last_img_path = img_level_path + imgs[-1]
                if counter < len(labels): #TODO: CHECK THIS OFF BY ONE BULLSHIT
                    if labels[counter] != -1:
                        print counter
                        valid_images.append(counter)
                        path = last_img_path.replace("/home/priyanka/cs229_emotion_detection/DemoDir/VGG_S_rgb/", "")
                        images.append(path)
                counter += 1
#                 if counter > 5: #because of bad internet
#                     return

grab_data('Emotion')
grab_data('cohn-kanade-images')
temp_labels = list()
for label in labels: #clean temp labels
    if label != -1:
        temp_labels.append(label)
labels = temp_labels


0
2
3
5
6
7
13
14
15
16
17
18
19
20
23
25
26
27
28
31
34
36
37
40
41
42
43
44
45
47
48
49
50
52
56
57
59
62
63
64
66
70
72
73
74
75
76
77
78
79
80
81
83
86
87
88
90
92
93
94
96
97
98
101
102
103
105
106
107
109
111
112
116
117
118
123
125
126
127
128
129
131
135
136
137
138
139
141
143
144
146
147
148
150
151
152
153
155
157
158
160
161
162
163
164
166
168
170
171
173
174
175
177
178
179
180
181
182
183
184
187
189
190
192
193
194
196
197
198
199
200
201
207
209
211
216
217
218
220
221
229
230
233
235
239
240
241
243
244
245
246
247
248
249
250
252
253
254
255
258
259
260
261
262
263
265
266
267
268
270
275
277
280
281
282
283
284
286
289
290
291
296
297
298
299
301
305
306
310
311
315
316
317
318
319
320
321
322
323
324
325
329
330
332
336
338
342
343
344
346
348
349
350
352
356
357
359
360
363
364
365
366
369
370
371
372
376
378
380
381
383
385
386
388
390
391
392
393
394
398
400
404
405
406
408
410
412
415
416
419
421
422
423
424
431
432
433
435
436
437
439
445
447
455
456
458
461
4

In [26]:
predictions = []
for image in images:
    input_image = caffe.io.load_image(os.path.join(DEMO_DIR,cur_net_dir,image))
    prediction = VGG_S_Net.predict([input_image],oversample=False)
    predictions.append(prediction)
    #print 'predicted category is {0}'.format(categories[prediction.argmax()])

predicted category is Angry
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Angry
predicted category is Angry
predicted category is Fear
predicted category is Fear
predicted category is Angry
predicted category is Sad
predicted category is Angry
predicted category is Disgust
predicted category is Neutral
predicted category is Neutral
predicted category is Neutral
predicted category is Neutral
predicted category is Neutral
predicted category is Neutral
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Sad
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Fear
predicted category is Angry
predicted category is Disgust
predicted category is Disgu

In [34]:
def make_predictions(predictions):
    prediction_num = []
    for i in range(0, len(predictions)):
        #print predictions[i]
        p = categories[predictions[i].argmax()]
        if p is 'Neutral':
            prediction_num.append(0)
        if p is 'Angry':
            prediction_num.append(1)
        if p is 'Disgust':
            prediction_num.append(3)
        if p is 'Fear':
            prediction_num.append(4)
        if p is 'Happy':
            prediction_num.append(5)
        if p is 'Sad':
            prediction_num.append(6)
        if p is 'Surprise':
            prediction_num.append(7)
    return prediction_num


327
327


In [35]:
def print_accuracy(labels_actual, labels_predicted):
    correct = 0
    wrong = 0
    for index in range(len(labels_actual)):
        if labels_actual[index] == labels_predicted[index]:
            correct += 1
        else:
            wrong += 1
    print 'num correct: ' + str(correct)
    print 'num wrong: ' + str(wrong)
    print 'accuracy: ' + str(float(correct)/(correct + wrong))

In [39]:
diff_labels = []
for label in labels:
    if label == 2:
        diff_labels.append(3)
    else: 
        diff_labels.append(label)

In [42]:
print diff_labels
print prediction_num

['3', '7', '1', '5', '5', '7', '7', '3', '5', '7', '6', '1', '4', '3', '7', '1', '5', '5', '7', '1', '7', '3', '5', '6', '1', '2', '4', '2', '2', '5', '4', '7', '3', '7', '7', '5', '6', '4', '3', '1', '7', '4', '7', '5', '7', '5', '6', '1', '4', '3', '4', '6', '7', '5', '3', '7', '5', '4', '7', '3', '5', '3', '7', '1', '7', '5', '5', '7', '6', '1', '5', '7', '3', '1', '2', '1', '4', '3', '7', '3', '7', '5', '6', '6', '7', '3', '4', '7', '5', '1', '5', '3', '7', '4', '5', '7', '3', '7', '1', '3', '5', '1', '4', '7', '4', '6', '5', '5', '7', '4', '1', '3', '7', '5', '3', '2', '3', '7', '1', '2', '3', '1', '7', '6', '5', '7', '5', '3', '5', '4', '7', '1', '7', '3', '4', '4', '7', '5', '3', '3', '5', '7', '3', '7', '7', '5', '1', '7', '6', '5', '1', '3', '7', '6', '1', '5', '2', '5', '7', '6', '1', '6', '1', '2', '3', '3', '5', '1', '7', '3', '7', '3', '1', '7', '4', '5', '7', '3', '2', '3', '7', '6', '2', '3', '7', '5', '3', '7', '6', '7', '6', '5', '6', '1', '2', '4', '1', '2', '4', '5',

In [43]:
print images[0]

cohn-kanade-images/S067/006/S067_006_00000011.png


In [54]:
cropped_images = []
for i in valid_images:
    path = 'CK_Cropped/' + str(i) + '.jpg'
    cropped_images.append(path)

In [55]:
print cropped_images

['CK_Cropped/0.jpg', 'CK_Cropped/2.jpg', 'CK_Cropped/3.jpg', 'CK_Cropped/5.jpg', 'CK_Cropped/6.jpg', 'CK_Cropped/7.jpg', 'CK_Cropped/13.jpg', 'CK_Cropped/14.jpg', 'CK_Cropped/15.jpg', 'CK_Cropped/16.jpg', 'CK_Cropped/17.jpg', 'CK_Cropped/18.jpg', 'CK_Cropped/19.jpg', 'CK_Cropped/20.jpg', 'CK_Cropped/23.jpg', 'CK_Cropped/25.jpg', 'CK_Cropped/26.jpg', 'CK_Cropped/27.jpg', 'CK_Cropped/28.jpg', 'CK_Cropped/31.jpg', 'CK_Cropped/34.jpg', 'CK_Cropped/36.jpg', 'CK_Cropped/37.jpg', 'CK_Cropped/40.jpg', 'CK_Cropped/41.jpg', 'CK_Cropped/42.jpg', 'CK_Cropped/43.jpg', 'CK_Cropped/44.jpg', 'CK_Cropped/45.jpg', 'CK_Cropped/47.jpg', 'CK_Cropped/48.jpg', 'CK_Cropped/49.jpg', 'CK_Cropped/50.jpg', 'CK_Cropped/52.jpg', 'CK_Cropped/56.jpg', 'CK_Cropped/57.jpg', 'CK_Cropped/59.jpg', 'CK_Cropped/62.jpg', 'CK_Cropped/63.jpg', 'CK_Cropped/64.jpg', 'CK_Cropped/66.jpg', 'CK_Cropped/70.jpg', 'CK_Cropped/72.jpg', 'CK_Cropped/73.jpg', 'CK_Cropped/74.jpg', 'CK_Cropped/75.jpg', 'CK_Cropped/76.jpg', 'CK_Cropped/77.jpg

In [56]:
predictions2 = []
for image in cropped_images:
    input_image = caffe.io.load_image(os.path.join(DEMO_DIR,cur_net_dir,image))
    prediction = VGG_S_Net.predict([input_image],oversample=False)
    predictions2.append(prediction)

IOError: [Errno 2] No such file or directory: './DemoDir/VGG_S_rgb/CK_Cropped/3.jpg'

In [58]:
import numpy as np
labels_cropped = np.loadtxt("ck_labels")

In [59]:
cropped_array = [0,
2,
4,
6,
7,
8,
9,
10,
11,
12,
13,
14,
15,
17,
18,
22,
23,
24,
25,
26,
29,
30,
31,
32,
34,
35,
36,
37,
38,
40,
42,
43,
47,
48,
49,
51,
54,
55,
56,
58,
60,
61,
63,
66,
70,
71,
72,
73,
74,
75,
77,
78,
80,
82,
84,
85,
86,
89,
90,
91,
93,
95,
96,
97,
98,
100,
101,
102,
103,
105,
106,
107,
108,
110,
113,
114,
117,
118,
119,
120,
123,
124,
125,
126,
127,
128,
129,
130,
132,
133,
134,
135,
136,
138,
139,
141,
142,
143,
144,
146,
147,
148,
149,
151,
153,
154,
155,
157,
158,
159,
160,
162,
163,
164,
167,
168,
169,
170,
171,
173,
174,
175,
180,
181,
182,
187,
188,
189,
191,
192,
194,
197,
198,
200,
201,
205,
206,
207,
212,
214,
217,
220,
221,
222,
224,
225,
229,
230,
231,
232,
236,
237,
241,
243,
246,
248,
249,
251,
252,
253,
254,
255,
256,
259,
260,
262,
263,
266,
267,
268,
269,
270,
271,
272,
273,
275,
276,
278,
279,
280,
283,
284,
286,
287,
292,
293,
294,
295,
297,
298,
299,
302,
304,
307,
308,
309,
312,
313,
314,
317,
318,
320,
325,
326,
332,
349,
351,
353,
355,
357,
361,
366,
367,
369,
372,
374,
375,
376,
383,
388,
389,
394,
395,
397,
402,
403,
408,
409,
412,
416,
417,
422,
423,
424,
426,
429,
437,
439,
444,
452,
456,
458,
461,
462,
463,
467,
468,
469,
470,
471,
472,
473,
476,
477,
479,
480,
481,
482,
484,
485,
486,
492,
494,
495,
496,
497,
499,
502,
504,
506,
507,
509,
510,
512,
515,
517,
518,
521,
522,
523,
524,
528,
529,
531,
533,
534,
535,
539,
540,
541,
544,
545,
547,
548,
549,
550,
551,
552,
553,
554,
555,
556,
557,
558,
559,
560,
561,
562,
563,
564,
565,
566,
567,
568,
569,
570,
571,
572,
573,
574,
575,
576,
577,
578,
579,
580,
581]

In [ ]:
predictions2 = []
for index in cropped_array:
    path = "CK_Cropped/" + str(index) + ".jpg"
    input_image = caffe.io.load_image(os.path.join(DEMO_DIR,cur_net_dir,path))
    prediction = VGG_S_Net.predict([input_image],oversample=False)
    predictions2.append(prediction)